In [1]:
from pathlib import Path
import glob
import re
import pandas as pd
import numpy as np


In [2]:
def natural_sort(l):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key=alphanum_key)

In [37]:
folder = Path(r'E:\project1_Manchester\CFD_Manchester\1_FullModel\Struct\1p5mm_v2_ManchesterInlet\Lines') 

# find all files 
files = natural_sort(glob.glob1(folder, '*.csv'))
file = files[-1000]

In [38]:
columns = ['Temperature (K)', 'Turbulent Kinetic Energy (J/kg)', 'TKE_Resolved',
       'TKE_Total', 'Velocity[i] (m/s)', 'Velocity[j] (m/s)',
       'Velocity[k] (m/s)', 'X (m)', 'Y (m)', 'Z (m)']

In [41]:
mean = pd.read_csv(folder / 'mean.csv')

In [42]:
df = pd.read_csv(folder/file)
df = df[columns]
x = df['X (m)']
y = df['Y (m)']
z = df['Z (m)']
df = (df - mean)**2
df

,Temperature (K),Turbulent Kinetic Energy (J/kg),TKE_Resolved,TKE_Total,Velocity[i] (m/s),Velocity[j] (m/s),Velocity[k] (m/s),X (m),Y (m),Z (m)
0,3.026019e-14,7.390937e-09,8.926664e-11,9.104720e-09,1.959959e-05,1.022523e-08,0.000123,5.009344e-31,4.911603e-31,0.000000e+00
1,3.083097e-14,9.030325e-09,1.064681e-10,1.109786e-08,1.423271e-05,7.268189e-08,0.000121,5.009344e-31,4.049268e-32,0.000000e+00
2,2.535916e-14,9.583287e-09,1.207964e-10,1.185594e-08,7.388934e-06,2.560931e-07,0.000113,5.009344e-31,2.496005e-31,0.000000e+00
3,3.665216e-15,1.226786e-09,1.258072e-10,2.138312e-09,4.414506e-07,8.109827e-07,0.000104,5.009344e-31,9.984021e-31,0.000000e+00
4,2.106084e-16,5.042227e-12,1.161219e-10,7.276938e-11,2.921217e-06,2.056092e-06,0.000085,5.009344e-31,3.561044e-31,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...
4373,1.617304e-11,2.147201e-07,3.407808e-07,1.449264e-08,1.166327e-03,2.133700e-02,0.021214,6.478828e-31,0.000000e+00,0.000000e+00
4374,1.617304e-11,2.147201e-07,3.407808e-07,1.449264e-08,1.166327e-03,2.133700e-02,0.021214,6.478828e-31,0.000000e+00,3.979754e-32
4375,6.843525e-11,4.805076e-08,1.518002e-07,2.903980e-08,5.803360e-03,1.425703e-02,0.007592,6.478828e-31,0.000000e+00,0.000000e+00
4376,6.843525e-11,4.805076e-08,1.518002e-07,2.903980e-08,5.803360e-03,1.425703e-02,0.007592,6.478828e-31,0.000000e+00,1.012317e-32


In [5]:
from tqdm.notebook import tqdm

In [43]:
for file in tqdm(files[-999:]):
    df_2 = pd.read_csv(folder/file)
    df = df + (df_2[columns] - mean)**2

df = np.sqrt(df / 1000)j

  0%|          | 0/999 [00:00<?, ?it/s]

In [45]:
df.to_csv(folder / 'std.csv', index=None)

In [34]:
# Filter 
loc = -0.147
df[(df['X (m)'] == loc) & (abs(df['Y (m)'])==0)]

,Temperature (K),Turbulent Kinetic Energy (J/kg),TKE_Resolved,TKE_Total,Velocity[i] (m/s),Velocity[j] (m/s),Velocity[k] (m/s),X (m),Y (m),Z (m)


In [36]:
def extract_line(df: pd.DataFrame, loc: float, dir: str):
    if dir == 'V':
        output = df[(df['X (m)'] == loc) & (abs(df['Y (m)'])==0)]
        assert len(output) <= 200
    elif dir =='H':
        output = df[(df['X (m)'] == loc) & (abs(df['Z (m)'])==0)]
        assert len(output) <= 200
    else:
        raise ValueError(f"Check dir: {dir}, should be 'V' or 'H' ")
    return output

In [37]:
for dir in ['V', 'H']:
    data = {}
    for loc in x.unique():
        data[loc] = extract_line(df, loc, dir)





In [38]:
data

{-0.105:       Temperature (K)  Turbulent Kinetic Energy (J/kg)  TKE_Resolved  \
 0          309.650028                         0.007628             0   
 1          309.650024                         0.006298             0   
 2          309.650019                         0.004886             0   
 3          309.650013                         0.003056             0   
 4          309.650009                         0.001800             0   
 ...               ...                              ...           ...   
 4044       309.650006                         0.001931             0   
 4045       309.650006                         0.001928             0   
 4046       309.650006                         0.001928             0   
 4047       309.650006                         0.001928             0   
 4146       309.650006                         0.001928             0   
 
       TKE_Total Reynolds Stress uu (J/kg) Reynolds Stress uv (J/kg)  \
 0      0.007628     1.79769313486232e+308

In [39]:
import pickle
parsed_data = r'C:\Users\yjw\Dropbox (MIT)\ResearchProjects\2020_CFD\DataProcessing\cfd_1_ManchesterLESVal\data\parsed_CFD_LineTables\mean.pkl'

with open(parsed_data, 'rb') as f:
    mean = pickle.load(f)
mean

array([[ 2.88000029e+02,  7.60555866e-03,  2.53353923e-04, ...,
        -1.05000000e-01,  5.34600000e-02,  0.00000000e+00],
       [ 2.88000024e+02,  6.32543030e-03,  2.72595389e-04, ...,
        -1.05000000e-01,  5.29200000e-02,  0.00000000e+00],
       [ 2.88000019e+02,  4.95836214e-03,  2.83614398e-04, ...,
        -1.05000000e-01,  5.23800000e-02,  0.00000000e+00],
       ...,
       [ 2.88000258e+02,  9.42343007e-04,  3.83864119e-02, ...,
        -1.47000000e-01,  0.00000000e+00,  9.18000000e-03],
       [ 2.88000258e+02,  9.42343007e-04,  3.83864119e-02, ...,
        -1.47000000e-01,  0.00000000e+00,  8.64000000e-03],
       [ 2.88000258e+02,  9.42343007e-04,  3.83864119e-02, ...,
        -1.47000000e-01,  0.00000000e+00,  8.10000000e-03]])

In [40]:
mean.shape

(4378, 17)